# Inside the Iceberg: Metadata structures

In the previous notebook, we saw Iceberg from the outside - creating tables, appending data, querying. Now let's look inside to understand exactly how Iceberg works.

We'll explore:

* **The catalog database**: What's actually in the SQLite file?
* **Metadata JSON files**: Schema, snapshots, and table history
* **Manifest files (AVRO)**: Lists of data files with statistics
* **Data files (Parquet)**: The actual data
* **The complete picture**: How a query uses all these pieces

By the end, you'll understand how Iceberg achieves atomic commits, time travel, and fast queries.

In [1]:
import json
import shutil
import daft
import pyarrow as pa
from pathlib import Path
from pyiceberg.catalog.sql import SqlCatalog
import sqlite3
from datetime import datetime

%reload_ext autoreload
%autoreload 2
from helpers import inspect_iceberg_table, inspect_metadata_json, inspect_manifest

## Setup: Create a table with history

First, let's create a table with multiple snapshots so we have interesting metadata to explore.

In [2]:
warehouse_path = Path('../data/warehouse_metadata').absolute()
shutil.rmtree(warehouse_path, ignore_errors=True)
warehouse_path.mkdir(parents=True, exist_ok=True)
catalog_db = warehouse_path / 'catalog.db'
catalog_db.unlink(missing_ok=True)

catalog = SqlCatalog(
    'metadata_demo',
    **{'uri': f'sqlite:///{catalog_db}', 'warehouse': f'file://{warehouse_path}'}
)
catalog.create_namespace('demo')
print("✅ Catalog initialized")

✅ Catalog initialized


In [3]:
# Load events data and create table with multiple operations
df_events = daft.read_json('../data/input/events.jsonl')

# Snapshot 1: Initial data
print("Snapshot 1: Creating initial load...")
df_batch1 = df_events.limit(30000)
arrow_table = df_batch1.to_arrow()
events_table = catalog.create_table('demo.events', schema=pa.schema(arrow_table.schema))
events_table.append(arrow_table)
print(f"Snapshot 1: Appended {len(arrow_table):,} records")

# Snapshot 2: Append more
print("Snapshot 2: Appending more data...")
df_batch2 = df_events.offset(30000).limit(30000)
arrow_table = df_batch2.to_arrow()
events_table.append(arrow_table)
print(f"Snapshot 2: Appended {len(arrow_table):,} more records")

# Snapshot 3: Delete some records
print("Snapshot 3: Deleting OperationMode events...")
events_table.delete("type = 'OperationMode'")
print("Snapshot 3: Deleted OperationMode events")

print(f"\n✅ Created table with {len(events_table.history())} snapshots")

/Users/eickler/Documents/knee-deep-in-the-lake/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Snapshot 1: Creating initial load...
[00:00] 🗡️ 🐟 Json Scan: 30,000 rows out, 0 B read | 🗡️ 🐟 Limit 30000: 30,000 rows in, 30,000 rows out

Snapshot 1: Appended 30,000 records
Snapshot 2: Appending more data...
[00:00] 🗡️ 🐟 Json Scan: 60,000 rows out, 0 B read | 🗡️ 🐟 Limit 30000: 60,000 rows in, 30,000 rows out

Snapshot 2: Appended 30,000 more records
Snapshot 3: Deleting OperationMode events...
Snapshot 3: Deleted OperationMode events

✅ Created table with 3 snapshots


## The catalog database

The catalog database is the **entry point** to all Iceberg tables. It's in our case a simple SQLite database that stores:

* **Table locations**: Where each table's metadata lives
* **Namespace properties**: Configuration for database schemas
* **Atomic pointers**: Current metadata file for each table

### Why use a catalog?

The catalog enables **atomic commits**. When a writer updates a table, the catalog

1. Writes a new metadata JSON file
2. Updates the catalog pointer atomically (SQL UPDATE)
3. In case of conflicts, returns an error and the client has to retry (optimistic concurrency)

The catalog is the **single source of truth** for which metadata file is current.

### Inspecting the catalog

Let's look inside the SQLite database:

In [4]:
conn = sqlite3.connect(catalog_db)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print("Tables in catalog database:")
for table in tables:
    print(f"  • {table[0]}")

Tables in catalog database:
  • iceberg_tables
  • iceberg_namespace_properties


In [5]:
# Show the schema of iceberg_tables
cursor.execute("PRAGMA table_info(iceberg_tables)")
columns = cursor.fetchall()
print("Schema of 'iceberg_tables':")
for col in columns:
    print(f"  {col[1]}: {col[2]}")

cursor.execute("PRAGMA table_info(iceberg_namespace_properties)")
columns = cursor.fetchall()
print("\nSchema of 'iceberg_namespace_properties':")
for col in columns:
    print(f"  {col[1]}: {col[2]}")


Schema of 'iceberg_tables':
  catalog_name: VARCHAR(255)
  table_namespace: VARCHAR(255)
  table_name: VARCHAR(255)
  metadata_location: VARCHAR(1000)
  previous_metadata_location: VARCHAR(1000)

Schema of 'iceberg_namespace_properties':
  catalog_name: VARCHAR(255)
  namespace: VARCHAR(255)
  property_key: VARCHAR(255)
  property_value: VARCHAR(1000)


In [6]:
cursor.execute("SELECT * FROM iceberg_tables")
rows = cursor.fetchall()

print("Registered Iceberg tables")
for row in rows:
    catalog_name, namespace, table_name, metadata_location, prev_metadata = row
    print(f"{namespace}.{table_name}")
    print(f"  Current metadata: {Path(metadata_location).name}")
    if prev_metadata:
        print(f"  Previous metadata: {Path(prev_metadata).name}")
    print()

cursor.execute("SELECT * FROM iceberg_namespace_properties")
rows = cursor.fetchall()

print("Registered namespaces:")
for row in rows:
    catalog_name, namespace, key, value = row
    print(f"{catalog_name}.{namespace}")
    print(f"  {key}: {value}")

Registered Iceberg tables
demo.events
  Current metadata: 00003-f1ee88cc-80d7-42d4-8976-14c518ac1ab6.metadata.json
  Previous metadata: 00002-9517bbd1-e634-41b4-bbde-52f3745baa21.metadata.json

Registered namespaces:
metadata_demo.demo
  exists: true


### What we see

The `iceberg_tables` table has:

* **metadata_location**: Points to the **current** metadata JSON file
* **previous_metadata_location**: Points to the **previous** metadata JSON file

This is how Iceberg achieves **atomic commits**:

```sql
UPDATE iceberg_tables
SET metadata_location = 'new_metadata.json',
    previous_metadata_location = 'old_metadata.json'
WHERE table_name = 'events'
  AND metadata_location = 'old_metadata.json'  -- Optimistic lock!
```

If two writers try to commit at the same time:
- First succeeds (updates the row)
- Second fails (WHERE clause doesn't match anymore)
- Second must retry with the new metadata

This is **optimistic concurrency control**!

In [7]:
conn.close()

## Metadata JSON files

Each commit creates a **new metadata JSON file**. This file contains:

* **Schema versions**: All schema versions (for time travel)
* **Partition specs**: All partition specs (for partition evolution)
* **Snapshots**: All snapshots with their manifest lists
* **Snapshot log**: Chronological list of snapshots
* **Current snapshot ID**: Pointer to the current snapshot
* **Metadata log**: History of metadata files

Let's find and inspect a metadata JSON file. 

In [8]:
# Find metadata files
table_dir = Path(events_table.location().replace('file://', ''))
metadata_files = sorted(table_dir.glob('metadata/*.metadata.json'))

print(f"Found {len(metadata_files)} metadata file(s):")
for i, mf in enumerate(metadata_files, 1):
    size = mf.stat().st_size
    print(f"  {i}. {mf.name} ({size:,} bytes, {size/1024:.1f} KB)")

# Use the latest metadata file
latest_metadata = metadata_files[-1]
print(f"\nUsing latest: {latest_metadata.name}")

Found 4 metadata file(s):
  1. 00000-5bc5fc7b-3c5e-417a-8f48-e403a5746cbd.metadata.json (1,000 bytes, 1.0 KB)
  2. 00001-50a91950-d66c-4fa6-b264-caa9191b9462.metadata.json (1,936 bytes, 1.9 KB)
  3. 00002-9517bbd1-e634-41b4-bbde-52f3745baa21.metadata.json (2,824 bytes, 2.8 KB)
  4. 00003-f1ee88cc-80d7-42d4-8976-14c518ac1ab6.metadata.json (3,802 bytes, 3.7 KB)

Using latest: 00003-f1ee88cc-80d7-42d4-8976-14c518ac1ab6.metadata.json


There is a lot of stuff in the file. Here is a visualization of the contents:

In [9]:
inspect_metadata_json(latest_metadata)

Format Version,2
Table UUID,75531e0f-6819-4cd6-9563-b775a436161c
Location,file:///Users/eickler/Documents/knee-deep-in-the-lake/02_iceberg/../data/warehouse_metadata/demo/events
Last Updated,2026-02-21 22:37:32
Current Snapshot,7283462870554044909
Timestamp:,2026-02-21 22:37:31
Manifest List:,snap-13305258960365828-0-7a04fadb-197c-48fb-b3dc-f94ba0183518.avro
Schema ID:,0
Summary:,added-data-files: 1added-files-size: 715559added-records: 30000operation: appendtotal-data-files: 1total-delete-files: 0total-equality-deletes: 0total-files-size: 715559total-position-deletes: 0total-records: 30000
Timestamp:,2026-02-21 22:37:31
Manifest List:,snap-710793980391093520-0-f4e8e102-6e75-47d0-b4ce-eafcc890746f.avro


### Schemas

Iceberg stores **all schema versions** in the metadata. Each schema has a unique ID.

When you read a snapshot, Iceberg uses the schema that was current at that snapshot. This enables:
* **Time travel with old schemas**
* **Schema evolution without rewrites**

### Snapshots

Each snapshot represents a **commit** to the table. Snapshots contain:

* **snapshot-id**: Unique identifier
* **timestamp-ms**: When this snapshot was created
* **manifest-list**: Path to AVRO file listing manifests
* **schema-id**: Which schema version to use
* **summary**: Statistics (operation, files added/deleted, records added/deleted)

### Logs

In addition (not shown):

* The `snapshot-log` is a chronological list of snapshots with timestamps. This enables:
  * **Time travel by timestamp**: "Show me data as of 2024-12-01"
  * **Audit trail**: When was each commit made?
* The `metadata-log` tracks which metadata files existed and when. This is used for:
  * **Metadata file expiration**: Clean up old metadata files
  * **Debugging**: Understand table history
  * **Consistency checks**: Verify metadata chain

The logs are kept separate because entries in `snapshots` may be cleaned up separately as we will see.

Can you guess from the metadata how the delete operation in the third snapshot was handled by Daft? 

Feel free to open the JSON file directly.

## Manifest Files (AVRO)

Manifests are the **index** that tells Iceberg which data files exist and where they are. The hierarchy is:

```
Snapshot
  └─ Manifest List (AVRO) ← Points to multiple manifests
       ├─ Manifest 1 (AVRO) ← Lists data files for partition 1
       ├─ Manifest 2 (AVRO) ← Lists data files for partition 2
       └─ Manifest N (AVRO) ← Lists data files for partition N
```

Each **manifest file** contains:
* **Data file paths**: Where the Parquet files are
* **Partition values**: What partition each file belongs to
* **Statistics**: Record counts, min/max values, null counts
* **File metadata**: Size, format, compression

This metadata enables **predicate pushdown** - skipping files without reading them.

### Finding Manifest Files

Manifest files are named with pattern: `<uuid>-m<N>.avro`

In [17]:
# Find manifest files
manifest_files = sorted(table_dir.glob('metadata/*-m*.avro'))
print(f"Found {len(manifest_files)} manifest file(s):")
for mf in manifest_files:
    size = mf.stat().st_size
    print(f"  • {mf.name} ({size:,} bytes)")

# Pick first manifest to inspect
if manifest_files:
    manifest_to_inspect = manifest_files[0]
    print(f"\nWill inspect: {manifest_to_inspect.name}")

Found 4 manifest file(s):
  • 7a04fadb-197c-48fb-b3dc-f94ba0183518-m0.avro (4,709 bytes)
  • ec659cb6-be8c-45c0-9ff6-d04c0136219a-m0.avro (5,069 bytes)
  • ec659cb6-be8c-45c0-9ff6-d04c0136219a-m1.avro (5,085 bytes)
  • f4e8e102-6e75-47d0-b4ce-eafcc890746f-m0.avro (4,713 bytes)

Will inspect: 7a04fadb-197c-48fb-b3dc-f94ba0183518-m0.avro


### Reading Manifest Files

Manifests are AVRO files. Let's read one and see what's inside.

If you don't have `fastavro` installed, run: `pip install fastavro`

In [18]:
try:
    import fastavro

    # Read the manifest
    with open(manifest_to_inspect, 'rb') as f:
        reader = fastavro.reader(f)
        records = list(reader)

    print(f"Manifest contains {len(records)} entry(ies)\n")

    # Show first entry in detail
    if records:
        entry = records[0]
        print("First entry structure:")
        print(f"  Status: {entry.get('status', 'N/A')}  (0=EXISTING, 1=ADDED, 2=DELETED)")

        data_file = entry.get('data_file', {})
        print(f"\n  Data file:")
        print(f"    Path: {Path(data_file.get('file_path', 'N/A')).name}")
        print(f"    Format: {data_file.get('file_format', 'N/A')}")
        print(f"    Records: {data_file.get('record_count', 0):,}")
        print(f"    Size: {data_file.get('file_size_in_bytes', 0):,} bytes")

        if data_file.get('value_counts'):
            print(f"\n    Value counts (first 3 columns):")
            for i, (col, count) in enumerate(data_file['value_counts'].items()):
                if i >= 3:
                    break
                print(f"      {col}: {count:,}")

        if data_file.get('lower_bounds'):
            print(f"\n    Lower bounds (first 2):")
            for i, (col, val) in enumerate(data_file['lower_bounds'].items()):
                if i >= 2:
                    break
                print(f"      {col}: {val!r}")

        if data_file.get('upper_bounds'):
            print(f"\n    Upper bounds (first 2):")
            for i, (col, val) in enumerate(data_file['upper_bounds'].items()):
                if i >= 2:
                    break
                print(f"      {col}: {val!r}")

except ImportError:
    print("⚠️  fastavro not installed. Install with: pip install fastavro")
    print("   We'll skip the detailed manifest inspection.")

⚠️  fastavro not installed. Install with: pip install fastavro
   We'll skip the detailed manifest inspection.


### Using the Helper Function

Let's use our helper to visualize the manifest:

In [19]:
if manifest_files:
    inspect_manifest(manifest_to_inspect)
else:
    print("No manifest files found")

⚠️  fastavro not installed. Install with: pip install fastavro


## Data Files (Parquet)

Finally, the actual data! Data files are standard **Parquet files**. Iceberg doesn't change Parquet - it just tracks them in manifests.

Key properties:
* **Immutable**: Once written, never modified
* **Referenced by manifests**: Manifests point to data files
* **Multiple files per table**: Each append creates new files
* **Deletes don't rewrite**: Delete files mark rows as deleted

Let's find and inspect a data file:

In [20]:
# Find data files
data_files = sorted(table_dir.glob('data/*.parquet'))
print(f"Found {len(data_files)} data file(s):")

total_size = 0
for df in data_files:
    size = df.stat().st_size
    total_size += size
    print(f"  • {df.name} ({size / 1024 / 1024:.2f} MB)")

print(f"\nTotal data size: {total_size / 1024 / 1024:.2f} MB")

Found 4 data file(s):
  • 00000-0-7a04fadb-197c-48fb-b3dc-f94ba0183518.parquet (0.68 MB)
  • 00000-0-ec659cb6-be8c-45c0-9ff6-d04c0136219a.parquet (0.51 MB)
  • 00000-0-f4e8e102-6e75-47d0-b4ce-eafcc890746f.parquet (0.63 MB)
  • 00000-1-ec659cb6-be8c-45c0-9ff6-d04c0136219a.parquet (0.48 MB)

Total data size: 2.31 MB


In [21]:
# Inspect first data file with PyArrow
if data_files:
    import pyarrow.parquet as pq

    data_file = data_files[0]
    pq_file = pq.ParquetFile(data_file)

    print(f"Inspecting: {data_file.name}\n")
    print(f"Total rows: {pq_file.metadata.num_rows:,}")
    print(f"Total columns: {pq_file.metadata.num_columns}")
    print(f"Row groups: {pq_file.metadata.num_row_groups}")
    print(f"Format version: {pq_file.metadata.format_version}")
    print(f"Created by: {pq_file.metadata.created_by}")

    print(f"\nSchema:")
    for i, field in enumerate(pq_file.schema):
        print(f"  {i+1}. {field.name}: {field.physical_type}")

Inspecting: 00000-0-7a04fadb-197c-48fb-b3dc-f94ba0183518.parquet

Total rows: 30,000
Total columns: 6
Row groups: 1
Format version: 2.6
Created by: parquet-cpp-arrow version 22.0.0

Schema:
  1. creationTime: INT64
  2. id: BYTE_ARRAY
  3. source: BYTE_ARRAY
  4. text: BYTE_ARRAY
  5. time: INT64
  6. type: BYTE_ARRAY


## The Complete Picture: Tracing a Query

Now let's trace how a query uses all these metadata structures:

```
SELECT * FROM events WHERE type = 'c8y_Event' AND time > '2024-01-01'
```

### Step-by-Step Query Execution

1. **Catalog Lookup** (SQLite)
   - Query: `SELECT metadata_location FROM iceberg_tables WHERE table_name = 'events'`
   - Result: Path to current metadata JSON file

2. **Read Metadata JSON**
   - Parse: `current-snapshot-id`
   - Find snapshot with that ID
   - Get: `manifest-list` path

3. **Read Manifest List** (AVRO)
   - Lists all manifest files for this snapshot
   - Each manifest covers a partition or set of files

4. **Read Manifests** (AVRO)
   - For each manifest, check statistics:
     - Does `lower_bounds['type']` ≤ 'c8y_Event' ≤ `upper_bounds['type']`?
     - Does `lower_bounds['time']` ≤ '2024-01-01' ≤ `upper_bounds['time']`?
   - If not: **skip this manifest entirely**
   - If yes: read the list of data files

5. **Predicate Pushdown on Files**
   - For each data file in relevant manifests:
     - Check file-level statistics
     - Skip files where predicates can't match

6. **Read Data Files** (Parquet)
   - Read only files that passed predicate pushdown
   - Within each file, read only necessary columns
   - Apply row-level filters

This is why Iceberg is fast - it reads minimal metadata to skip most of the data!

### Visualizing the Hierarchy

In [22]:
# Show the complete metadata hierarchy
print("Complete Iceberg Metadata Hierarchy:\n")
print("1. 📚 Catalog (SQLite)")
print(f"   {catalog_db.name}")
print(f"   └─ Table: demo.events → {latest_metadata.name}")
print()
print("2. 📄 Metadata JSON")
print(f"   {latest_metadata.name}")
print(f"   ├─ Schema: {len(metadata['schemas'])} version(s)")
print(f"   ├─ Partition specs: {len(metadata['partition-specs'])}")
print(f"   └─ Snapshots: {len(metadata['snapshots'])}")
print()
print("3. 📦 Manifest Files (AVRO)")
for i, mf in enumerate(manifest_files, 1):
    print(f"   {mf.name} ({mf.stat().st_size:,} bytes)")
print()
print("4. 💾 Data Files (Parquet)")
for i, df in enumerate(data_files, 1):
    print(f"   {df.name} ({df.stat().st_size / 1024 / 1024:.2f} MB)")
print()
print(f"Total metadata overhead: {sum(mf.stat().st_size for mf in metadata_files + manifest_files) / 1024:.1f} KB")
print(f"Total data size: {sum(df.stat().st_size for df in data_files) / 1024 / 1024:.2f} MB")

Complete Iceberg Metadata Hierarchy:

1. 📚 Catalog (SQLite)
   catalog.db
   └─ Table: demo.events → 00003-f1ee88cc-80d7-42d4-8976-14c518ac1ab6.metadata.json

2. 📄 Metadata JSON
   00003-f1ee88cc-80d7-42d4-8976-14c518ac1ab6.metadata.json
   ├─ Schema: 1 version(s)
   ├─ Partition specs: 1
   └─ Snapshots: 3

3. 📦 Manifest Files (AVRO)
   7a04fadb-197c-48fb-b3dc-f94ba0183518-m0.avro (4,709 bytes)
   ec659cb6-be8c-45c0-9ff6-d04c0136219a-m0.avro (5,069 bytes)
   ec659cb6-be8c-45c0-9ff6-d04c0136219a-m1.avro (5,085 bytes)
   f4e8e102-6e75-47d0-b4ce-eafcc890746f-m0.avro (4,713 bytes)

4. 💾 Data Files (Parquet)
   00000-0-7a04fadb-197c-48fb-b3dc-f94ba0183518.parquet (0.68 MB)
   00000-0-ec659cb6-be8c-45c0-9ff6-d04c0136219a.parquet (0.51 MB)
   00000-0-f4e8e102-6e75-47d0-b4ce-eafcc890746f.parquet (0.63 MB)
   00000-1-ec659cb6-be8c-45c0-9ff6-d04c0136219a.parquet (0.48 MB)

Total metadata overhead: 28.5 KB
Total data size: 2.31 MB


## Review Questions

Test your understanding:

--> What the relationship between the schema in the Parquet files and the schema in the Iceberg metadata?
--> Ids of the fields -- what happens if you add two columns with different Ids but same name?
how is the delete handled here?

1. **Why store metadata in multiple JSON files instead of one?**
   - Hint: Think about atomicity and append-only operations.

2. **What would happen if you directly edited a data file?**
   - Would the manifest notice? Would queries see your changes?

3. **How does Iceberg achieve atomic commits with SQLite?**
   - What SQL statement is used? What makes it atomic?

4. **Why separate manifest lists from manifest files?**
   - Why not put all data files in one manifest?

5. **How does predicate pushdown work?**
   - At what levels can files/partitions be skipped?

6. **What's the metadata overhead for this table?**
   - Calculate: metadata size / data size
   - Is this reasonable?

## Hands-on Challenge

--> What happens if the Iceberg schema and the Parquet schema do not match?

### Challenge 1: Parse Metadata Manually

1. Open the latest metadata JSON in a text editor
2. Find the `current-snapshot-id`
3. Locate that snapshot in the `snapshots` array
4. Extract the `manifest-list` path
5. Verify this file exists in the metadata directory

### Challenge 2: Analyze Manifest Statistics

1. Read a manifest file using fastavro
2. For each data file entry, extract:
   - Record count
   - File size
   - Lower/upper bounds for 'type' column
3. Calculate: total records, average file size

### Challenge 3: Simulate Predicate Pushdown

1. Write a query filter: `type = 'c8y_Measurement'`
2. Read manifests and check statistics
3. Count how many files would be skipped
4. Calculate: % of data skipped

Use the cells below:

In [23]:
# Challenge 1: Your code here


In [24]:
# Challenge 2: Your code here


In [25]:
# Challenge 3: Your code here


## Summary

In this deep dive, we explored:

* **Catalog Database**: The atomic pointer to current metadata
  - Enables optimistic concurrency control
  - Single UPDATE statement makes commits atomic

* **Metadata JSON**: Complete table state
  - All schema versions (for time travel)
  - All snapshots with manifest lists
  - Snapshot log for temporal queries
  - Metadata log for file management

* **Manifest Files**: Index of data files
  - AVRO format for efficiency
  - Per-file statistics for pruning
  - Partition information
  - Enables predicate pushdown

* **Data Files**: Immutable Parquet
  - Never modified after creation
  - Referenced by manifests
  - Standard Parquet format

### Key Insights

1. **Metadata is append-only**: New files created, old ones retained
2. **Catalog is the single source of truth**: Points to current metadata
3. **Statistics enable pruning**: Skip files/partitions without reading
4. **Everything is versioned**: Time travel works by reading old snapshots
5. **Minimal metadata overhead**: ~KB of metadata per GB of data

### What's Next?

Now that you understand the internal structures, we'll explore:
* **Time travel**: Using snapshots for historical queries
* **Schema evolution**: How column changes work in metadata
* **Concurrency**: Simulating optimistic locking conflicts
* **Partitioning**: Managing millions of files efficiently